In [6]:
!pip install --upgrade pip
!pip install pyarrow==10.0.1


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 2.6 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 23.0.1
    Uninstalling pip-23.0.1:
      Successfully uninstalled pip-23.0.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.9/22.9 MB 2.9 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 11.0.0
    Uninstalling pyarrow-11.0.0:
      Successfully uninstalled pyarrow-11.0.0


In [3]:
from faker import Faker
import time

fake = Faker()

while True:
    for i in range(50):
        print(f"Sensor {i+1} - Temperature: {fake.pyfloat(left_digits=2, right_digits=2, positive=True)}°C, Pressure: {fake.pyfloat(left_digits=2, right_digits=2, positive=True)}Pa, Humidity: {fake.pyfloat(left_digits=2, right_digits=2, positive=True)}%, Noise levels: {fake.pyfloat(left_digits=2, right_digits=2, positive=True)}dB")
    time.sleep(5)


Sensor 1 - Temperature: 5.64°C, Pressure: 31.86Pa, Humidity: 11.49%, Noise levels: 57.35dB
Sensor 2 - Temperature: 17.16°C, Pressure: 94.92Pa, Humidity: 34.86%, Noise levels: 92.47dB
Sensor 3 - Temperature: 3.82°C, Pressure: 80.72Pa, Humidity: 76.9%, Noise levels: 12.87dB
Sensor 4 - Temperature: 23.49°C, Pressure: 87.84Pa, Humidity: 93.79%, Noise levels: 88.5dB
Sensor 5 - Temperature: 23.31°C, Pressure: 76.33Pa, Humidity: 90.6%, Noise levels: 79.12dB
Sensor 6 - Temperature: 49.69°C, Pressure: 71.84Pa, Humidity: 89.61%, Noise levels: 45.65dB
Sensor 7 - Temperature: 53.0°C, Pressure: 29.0Pa, Humidity: 47.92%, Noise levels: 89.42dB
Sensor 8 - Temperature: 24.7°C, Pressure: 11.6Pa, Humidity: 4.76%, Noise levels: 94.97dB
Sensor 9 - Temperature: 3.97°C, Pressure: 67.69Pa, Humidity: 78.37%, Noise levels: 82.68dB
Sensor 10 - Temperature: 29.0°C, Pressure: 42.25Pa, Humidity: 43.4%, Noise levels: 66.23dB
Sensor 11 - Temperature: 34.26°C, Pressure: 22.5Pa, Humidity: 68.47%, Noise levels: 2.67dB
S

KeyboardInterrupt: 

In [4]:
import random
import json
import time
import pandas as pd
from snowflake.connector.pandas_tools import write_pandas
import snowflake.connector

In [6]:
# Define the list of sensors
sensors = [
    {"sensor_id": "1a2b3c4d", "sensor_name": "TemperatureSensor01", "sensor_type": "Temperature"},
    {"sensor_id": "2b3c4d5e", "sensor_name": "PressureSensor02", "sensor_type": "Pressure"},
    {"sensor_id": "3c4d5e6f", "sensor_name": "HumiditySensor03", "sensor_type": "Humidity"},
    {"sensor_id": "4d5e6f7g", "sensor_name": "VibrationSensor04", "sensor_type": "Vibration"},
    {"sensor_id": "5e6f7g8h", "sensor_name": "NoiseSensor05", "sensor_type": "Noise"},
    {"sensor_id": "6f7g8h9i", "sensor_name": "RadiationSensor06", "sensor_type": "Radiation"},
    {"sensor_id": "7g8h9i10", "sensor_name": "DustDensitySensor07", "sensor_type": "Dust Density"},
    {"sensor_id": "8h9i10j1", "sensor_name": "GasSensor08", "sensor_type": "Gas Concentration"},
    {"sensor_id": "9i10j1k2", "sensor_name": "ProximitySensor09", "sensor_type": "Proximity"},
    {"sensor_id": "10j1k2l3", "sensor_name": "FlowRateSensor10", "sensor_type": "Flow Rate"}
]


In [19]:

# Define the range of the values
values_range = {
    "Temperature": ((0, 50), "Celsius"),  
    "Pressure": ((950, 1050), "hPa"),  
    "Vibration": ((0, 10), "mm/s"), 
    "Humidity": ((0, 100), "%"),  
    "Noise": ((0, 100), "dB"), 
    "Radiation": ((0, 20), "mSv"),  
    "Dust Density": ((0, 500), "µg/m3"), 
    "Gas Concentration": ((0, 100), "ppm"),  
    "Proximity": ((0, 10), "m"), 
    "Flow Rate": ((0, 100), "L/min")
}

def generate_sensor_data(sensor):
    """
    Generate a mock reading for a sensor.
    """
    timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
    sensor_type = sensor["sensor_type"]
    value_range, unit = values_range[sensor_type]
    
    value = round(random.uniform(*value_range), 2)
    
    return {
        "sensor_id": sensor["sensor_id"],
        "sensor_name": sensor["sensor_name"],
        "sensor_type": sensor_type,
        "value": value,
        "measurement_unit": unit,
        "timestamp": timestamp
    }

# Function to connect to Snowflake and upload the data
def upload_to_snowflake(df):
    conn = snowflake.connector.connect(
        user=SNOWFLAKE_USER,
        password=SNOWFLAKE_PASSWORD,
        account=SNOWFLAKE_ACCOUNT,
        warehouse=SNOWFLAKE_WAREHOUSE,
        database=SNOWFLAKE_DATABASE,
        schema=SNOWFLAKE_SCHEMA,
        role=SNOWFLAKE_ROLE
    )

    success, nchunks, nrows, _ = write_pandas(conn, df, SNOWFLAKE_TABLE)
    print(f"Successfully uploaded {nrows} rows to Snowflake.")

data_pool = []
start_time = time.time()

while True:
    for sensor in sensors:
        data = generate_sensor_data(sensor)
        data_pool.append(data)
        
    elapsed_time = time.time() - start_time
    if elapsed_time > 10:  # 300 seconds = 5 minutes
        df = pd.DataFrame(data_pool)
        upload_to_snowflake(df)
        data_pool = []
        start_time = time.time()

    time.sleep(5)  # Sleep for five seconds before generating the next set of data

Successfully uploaded 30 rows to Snowflake.
Successfully uploaded 20 rows to Snowflake.
Successfully uploaded 20 rows to Snowflake.
Successfully uploaded 20 rows to Snowflake.
Successfully uploaded 20 rows to Snowflake.
Successfully uploaded 20 rows to Snowflake.
Successfully uploaded 20 rows to Snowflake.
Successfully uploaded 20 rows to Snowflake.


KeyboardInterrupt: 

In [7]:
from faker import Faker
import time
import snowflake.connector

fake = Faker()

conn = snowflake.connector.connect(
        user=SNOWFLAKE_USER,
        password=SNOWFLAKE_PASSWORD,
        account=SNOWFLAKE_ACCOUNT,
        warehouse=SNOWFLAKE_WAREHOUSE,
        database=SNOWFLAKE_DATABASE,
        schema=SNOWFLAKE_SCHEMA,
        role=SNOWFLAKE_ROLE
)

while True:
    data = []
    for i in range(50):
        data.append((f"Sensor {i+1}", fake.pyfloat(left_digits=2, right_digits=2, positive=True), fake.pyfloat(left_digits=2, right_digits=2, positive=True), fake.pyfloat(left_digits=2, right_digits=2, positive=True), fake.pyfloat(left_digits=2, right_digits=2, positive=True)))
    cursor = conn.cursor()
    cursor.executemany("INSERT INTO your-table (sensor_name, temperature, pressure, humidity, noise_levels) VALUES (%s,%s,%s,%s,%s)", data)
    conn.commit()
    time.sleep(60)


ProgrammingError: 001003 (42000): SQL compilation error:
syntax error line 1 at position 16 unexpected '-'.
syntax error line 1 at position 23 unexpected '('.